# Visualising GEE `Feature` objects

In [1]:
import os
import ee
import json
import togeojsontiles
import ipywidgets
from IPython.display import Image
import folium

ee.Initialize()

The purpose of this notebook is to demonstrate how to visualise Google Earth Engine `Feature` data (i.e. GeoJSON tables) for exploratory purposes. The workflow below provides a method for filtering attributes of interest and generating tile images that can be visualised from the notebook. 

## Functions

In [2]:
# Function to plot tiles
def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url

In [3]:
## -- filter shapefile to feature of interest
# Currently tailored to MoL's data scheme
# This function filters GeoJSON by 2 attributes
def filterShp(shp,sp,facet):
    filteredShape = shp.filter(ee.Filter.eq('ptype',facet)).filter(ee.Filter.eq('taxa', sp)).filter(ee.Filter.neq('pvalue', None))
    return (filteredShape)

## ---return min and max vlaues (for styling)
def extractStats(filtered_shape,column):
    stats  = filtered_shape.reduceColumns(
    selectors = [column],
    reducer = ee.Reducer.minMax())
    return(stats)

## --- create images out of the filtered shapefiles
def toImage(filtered_shape,column,lyr_name):
    image = filtered_shape.reduceToImage(
    properties = [column],
    reducer = ee.Reducer.first())
    image = image.rename([lyr_name])
    return(image)


##  Load and filter GeoJSON data 

In [4]:
# Load datasets
mol_facets = ee.FeatureCollection("users/guizarcou/mol_patterns_facet_map")
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")

In [5]:
# Get mammals richness
sr_mammals = filterShp(mol_facets,'mammals','sr')
sr_mammals

In [6]:
# Reduce to image
sr_mammals_image = toImage(sr_mammals,'pvalue','sr_mammals')
sr_mammals_image

In [8]:
# get stats (min and max for styling)
sr_mammals_stats = extractStats(sr_mammals,'pvalue')

In [9]:
# Generate palette
spectralPal = ['#9e0142','#d53e4f','#f46d43','#fdae61','#fee08b','#e6f598','#abdda4','#66c2a5','#3288bd','#5e4fa2']
spectralPal.reverse()

# Define visParams for the filtered table
sr_mammals_visParams = sr_mammals_image.visualize(
        min = sr_mammals_stats.get('min'), 
        max = sr_mammals_stats.get('max'),
        palette = spectralPal,
        opacity =  0.7,
      )

# Define visParams
wdpa_visParams = {'color': '#00e5e5', 'opacity':0.6}

In [10]:
# Generate tiles
tiles_sr_mammals = tile_url(sr_mammals_visParams)
tiles_wdpa = tile_url(wdpa,wdpa_visParams)

## Plot the results

In [11]:
map = folium.Map(tiles='Mapbox Bright')
map.add_tile_layer(tiles=tiles_sr_mammals, attr="Map of Life")
map.add_tile_layer(tiles=tiles_wdpa, attr="WDPA-WCMC") 
map